# <center>Block 4: Discrete optimal transport</center>
### <center>Alfred Galichon (NYU)</center>
## <center>`math+econ+code' masterclass on matching models, optimal transport and applications</center>
<center>© 2018-2019 by Alfred Galichon. Support from NSF grant DMS-1716489 is acknowledged. James Nesbit contributed.</center>

## Optimal Transport I: Discrete Transport

### Learning Objectives

* Optimal assignment problem

* Pairwise stability, Walrasian equilibrium

* Computation

### References

* [OTME], Ch. 3

* Roth, Sotomayor(1990). *Two-Sided Matching*. Cambridge.

* Koopmans and Beckmann (1957). "Assignment problems and the location of economic activities." *Econometrica*.

* Shapley and Shubik (1972). The assignment game I: The core." *IJGT*.

* Becker (1993). *A Treatise of the Family*. Harvard.

* Gretsky, Ostroy, and Zame (1992). "The nonatomic assignment model." *Economic Theory*.

* Burkard, Dell'Amico, and Martello (2012). *Assignment Problems*. SIAM.

* Dupuy and Galichon (2014). "Personality traits and the marriage market." *JPE*.

## Motivation

### Optimal Transport

Consider the problem of assigning a possibly infinite number of workers
and firms.

* Each worker should work for one firm, and each firm should hire one worker.

* Workers and firms have heterogenous characteristics; let $x\in \mathcal{X}$ and $y\in\mathcal{Y}$ be the characteristics of workers and firms respectively.

* Workers and firms are in equal mass, which is normalized to one. The distribution of worker's types is $P$, and the distribution of the firm's types is $Q$, where $P$ and $Q$ are probability measures on $\mathcal{X}$ and $\mathcal{Y}$.

It is assumed that if a worker $x$ matches with a firm $y$, the total output generated is $\Phi_{xy}$. The questions are then:

* **Optimality:** what is the optimal assignment in the sense that it maximizes the overal output generated?

* **Equilibrium:** what are the equilibrium assignment and the equilibrium wages?

* **Efficiency:** do these two notions coincide?

The same tools have been used by Gary Becker to study the heterosexual marriage market, where $x$ is the man's characteristics, and $y$ is the woman's characteristics, and "wages" are replaced by "transfers".

## Data

In this block, we shall take a first look at marriage data (while a worker-firm example will be seen in next block). Dupuy and Galichon (JPE, 2014) study a marriage dataset where, in addition to usual socio-demographic variables (such as education and age), measures of personality traits are reported.

* The literature on quantitative psychology argues that one can capture relatively well an individual's personality along five dimensions, the "big 5" - consciousness, extraversion, agreableness, emotional stability, autonomy - assessed though a standardized questionaire.

* In addition to this, we observed a (self-assessed) measure of health, risk-aversion, education, height and body mass index = weight in kg/ (height in m)$^2$. In total, the available characteristics $x_{i}$ of man $i$ and $y_{j}$ of woman $j$ are both $10$-dimensional vectors.

* It is assumed that the surplus of interaction is given by $\Phi\left(x_{i},y_{j}\right)  =x_{i}^{\intercal}Ay_{j}$, where $A$ is a *given* $10 \times 10$ matrix. (later in this course, we'll see how to estimate $A$ based on matched marital data).

Today, we solve a central planner's problem (a stylized version of the problem OKCupids would solve): given a population of men and a population of women, how do we mutually assign these in order to:

1. maximize matching surplus 

2. attain a (hopefully) stable assignment.

In [1]:
library('Matrix')
library('gurobi')
thePath = getwd()
nbcar = 10 # number of characteristics

data_X = as.matrix(read.csv(paste0(thePath,"/Xvals.csv"), sep=",", header=TRUE)) # loads the data
Xvals = matrix(as.numeric(data_X[,1:nbcar]), ncol=nbcar)

data_Y = as.matrix(read.csv(paste0(thePath,"/Yvals.csv"), sep=",", header=TRUE)) # loads the data
Yvals = matrix(as.numeric(data_Y[,1:nbcar]), ncol=nbcar)

data_aff = as.matrix(read.csv(paste0(thePath,"/affinitymatrix.csv"),sep=",", header=TRUE)) # loads the data

Loading required package: slam
Warning message in file(file, "rt"):
"cannot open file 'C:/Users/jmcgn/Dropbox/MEC_optim_jan2019/slides_ipynb_mec_optim/Xvals.csv': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection


In [37]:
head(data_X)

educm,heightm,BMIm,healthm,consm,extram,agreem,emom,autom,riskym
2,186,28.90508,3,-0.752877,-0.360787,-0.711276,-0.291031,0.840217,0.479437
2,176,27.44060,3,0.345542,-0.805524,-0.251796,-0.305475,-0.064454,0.030303
3,187,23.16337,3,-0.759678,0.898007,-0.029462,-0.672859,-0.961691,-0.556598
1,184,29.24149,2,-0.455688,-1.053375,-0.041612,0.436133,0.121873,0.992084
1,174,23.78121,4,-1.440239,1.163730,0.293750,-0.538922,0.782285,-1.401034
1,186,21.96786,3,-1.008298,-0.484221,1.155301,0.267899,0.927354,0.011056


In [38]:
head(data_Y)

educv,heightv,BMIv,healthv,consv,extrav,agreev,emov,autov,riskyv
2,159,22.94213,4,-0.352262,0.065096,-0.713136,-0.529817,-0.066740,0.271632
2,165,22.03857,3,-0.741707,-0.484221,0.219906,0.706937,0.685428,0.353834
3,170,20.76125,3,0.327571,-0.180299,1.052070,0.999001,0.177472,-0.201117
2,160,22.65625,4,-0.187640,-1.299261,1.223071,0.154011,2.284336,-0.172070
2,165,22.77319,3,0.078951,0.613921,0.122749,0.073875,-0.253068,0.042352
1,168,19.13265,3,-1.429069,0.472616,1.240802,0.695631,1.163253,-1.445496


In [39]:
head(data_aff)

X,educw,heightw,BMIw,healthw,consw,extraw,agreew,emow,autow,riskyw
educm,0.56,0.02,-0.08,0.02,-0.04,-0.01,-0.03,-0.04,0.05,-0.02
heightm,0.01,0.18,0.04,-0.01,-0.04,0.05,0.02,0.02,0.02,0.02
BMIm,-0.05,0.05,0.21,0.01,0.06,0.00,-0.04,0.04,-0.01,0.01
healthm,-0.07,0.00,-0.06,0.14,-0.04,0.05,-0.04,0.04,0.02,0.00
consm,-0.06,-0.03,0.07,0.00,0.14,0.07,0.04,0.06,-0.02,-0.01
extram,0.01,-0.02,0.05,0.02,-0.06,0.02,-0.02,-0.01,-0.03,-0.05


## The Discrete Monge-Kantorovich Theorem

### Discrete case
Assume that the type spaces $\mathcal{X}$ and $\mathcal{Y}$ are finite, so $\mathcal{X=}\left\{  1,...,N\right\}  $, and $\mathcal{Y}=\left\{1,...,M\right\}$.

The total mass of workers and jobs is normalized to one. The mass of workers of type $x$ is $p_{x}$; the mass of jobs of type $y$ is $q_{y}$, with $\sum_{x}p_{x}=\sum_{y}q_{y}=1$.

Let $\pi_{xy}$ be the mass of workers of type $x$ assigned to jobs of type $y$. Every worker is busy and every job is filled, thus

<a name="dicsr-constraints"></a>
\begin{align*}
\sum_{y\in\mathcal{Y}}\pi_{xy}=p_{x}\text{ and }\sum_{x\in\mathcal{X}}\pi
_{xy}=q_{y}. 
\end{align*}

(Note that this formulation allows for mixing, i.e. it allows for $\pi_{xy}>0$ and $\pi_{xy^{\prime}}>0$ to hold simultaneously with $y\neq y^{\prime}$.) The set of $\pi\geq0$ satisfying [the discrete contraints](#dicsr-constraints) is denoted by

\begin{align*}
\pi\in\mathcal{M}\left(p,q\right).
\end{align*}

Assume the economic output created when assigning worker $x$ to job $y$ is $\Phi_{xy}$. Hence, under assignment $\pi$, the total output is $\sum _{xy}\pi_{xy}\Phi_{xy}$.

Thus, the optimal assignment is

<a name="OAP"></a>
\begin{align*}
\max_{\pi\geq0}  &  \sum_{xy}\pi_{xy}\Phi_{xy} \\
s.t.~  &  \sum_{y\in\mathcal{Y}}\pi_{xy}=p_{x} \left[u_{x}\right] \\
&  \sum_{x\in\mathcal{X}}\pi_{xy}=q_{y}~\left[v_{y}\right]
\end{align*}

and it is now a finite-dimensional linear programming problem.

Note that it is nothing else than the Monge-Kantorovich problem when $P$ and $Q$ are discrete probability measures on $\mathcal{X=}\left\{1,...,N\right\}  $, and $\mathcal{Y}=\left\{  1,...,M\right\}$.

---
**Theorem**
1. The value of the [primal problem](#OAP) *coincides with the value of the dual problem*

    <a name="dual-discr"></a>
    \begin{align*}
    \min_{u,v}  &  \sum_{x\in\mathcal{X}}p_{x}u_{x}+\sum_{y\in\mathcal{Y}}
    q_{y}v_{y}\\
    s.t.  &  u_{x}+v_{y}\geq\Phi_{xy}~\left[\pi_{xy}\geq0\right]
    \end{align*}

2. Both the primal and the dual problems have optimal solutions. If $\pi$ *is a solution to the primal problem* and $\left(u,v\right)$ is *a solution to the dual problem, then by complementary slackness*,

    <a name="complSlack"> </a>
    \begin{align*}
    \pi_{xy}>0\text{ implies }u_{x}+v_{y}=\Phi_{xy}.
    \end{align*}

---

---
**Proof**
The proof follows from the min-cost flow duality result, but let us rewrite it anyway. 

1. The value of the [primal problem](#OAP) can be written as $\max_{\pi\geq0}\min_{u,v}S\left(  \pi,u,v\right)$, where

    \begin{align*}
    S\left(  \pi,u,v\right)  :=\sum_{xy}\pi_{xy}\Phi_{xy}+\sum_{x\in\mathcal{X}}u_{x}(p_{x}-\sum_{y\in\mathcal{Y}}\pi_{xy})+\sum_{y\in\mathcal{Y}}v_{y}(q_{y}-\sum_{x\in\mathcal{X}}\pi_{xy})
    \end{align*}

    but by the minmax theorem, this value is equal to $\min_{u,v}\max_{\pi\geq 0}S\left(  \pi,u,v\right)$, which is the value of the [dual problem](#ual-discr}).

2. Follows by noting that, for a primal solution $\pi$ and a dual solution $\left(  u,v\right)  $, then $S\left( \pi,u,v\right)  =\sum_{xy}\pi_{xy} \Phi_{xy}$. $\blacksquare$

---

Note that this result is the min-cost flow duality theorem in the bipartite case, as seen in lecture $2$, after setting transportation cost through $xy\in\mathcal{X}\times\mathcal{Y}$ to $c_{xy}=-\Phi_{xy}$, and $n_{t}=-p_{t}1\left\{  t\in\mathcal{X}\right\}  +q_{t}\mathbf{1}\left\{  t\in\mathcal{Y}\right\}$. We see various new interpretations of the result.

The following statements are equivalent:

* $\pi$ is an optimal solution to the primal problem, and $\left(
u,v\right)  $ is an optimal solution to the dual problem, and

* (i) $\pi\in M\left(  p,q\right)  $

* (ii) $u_{x}+v_{y}\geq\Phi_{xy}$

* (iii) $\pi_{xy}>0$ implies $u_{x}+v_{y}\leq\Phi_{xy}$.

We saw the direct implication. But the converse is easy: take $\pi$ and $\left(  u,v\right)  $ satisfying (i)--(iii), Then one has

\begin{align*}
dual\leq\sum_{x}p_{x}u_{x}+\sum_{y}q_{y}v_{y}=\sum_{xy}\pi_{xy}\left(
u_{x}+v_{y}\right)  \leq\sum_{xy}\pi_{xy}\Phi_{xy}\leq primal
\end{align*}

but by the MK duality theorem, both ends coincide. Thus $\pi$ is optimal for the primal and $\left(  u,v\right)  $ for the dual.

### Unassigned Agents

A important variant of the problem exists with $\sum_{x\in\mathcal{X}}p_{x}\neq\sum_{y\in\mathcal{Y}}q_{y}$ and the primal constraints become inequality constraints. The duality then becomes

\begin{align*}
\begin{array}
[c]{rrr}
\max_{\pi\geq0}\sum\pi_{xy}\Phi_{xy} & = & \min_{u,v}\sum_{x\in\mathcal{X}
}p_{x}u_{x}+\sum_{y\in\mathcal{Y}}q_{y}v_{y}\\
s.t.~\sum_{y\in\mathcal{Y}}\pi_{xy}\leq p_{x} &  & u\geq0,~v\geq0 \\
\sum_{x\in\mathcal{X}}\pi_{xy}\leq q_{y} &  & u_{x}+v_{y}\geq\Phi_{xy}
\end{array}
\end{align*}

### Pairwise stability

In a marriage context, an important concept is stability:

An outcome is a vector $\left(  \pi,u,v\right)  $, where $u_{x}$ and $v_{y}$ are $x$'s and $y$'s payoffs, and $\pi$ is a matching that is

<a name="primFeas"></a>
\begin{align*}
\pi\in\mathcal{M}\left(  p,q\right).
\end{align*}

A pair $xy$ is blocking if $x$ and $y$ can find a way of sharing their joint surplus $\Phi_{xy}$ in such a way that $x$ gets more than $u_{x}$ and $y$ gets more than $v_{y}$. Hence there is no blocking pair if and only if for every $x$ and $y$, one has

<a name="noBlocking"></a>
\begin{align*}
u_{x}+v_{y}\geq\Phi_{xy}.
\end{align*}

If $x$ and $y$ are actually matched, their utilities $u_{x}$ and $v_{y}$ need to be feasible, i.e. the above inequality should be saturated. Hence

<a name="cplSlck"></a>
\begin{align*}
\pi_{xy}>0\text{ implies }u_{x}+v_{y}=\Phi_{xy}.
\end{align*}

---
**Definition:**

A matching that satisfies [primal feasbilitity](#primFeas), [no blocking](#noBlocking), and [complementary slackness](#cplSlck) is called a stable matching.

As it turns out, these conditions are precisely the conditions that express complementarity slackness in the Monge-Kantorovich problem. Therefore, outcome$\left(  \pi,u,v\right)  $ is stable if and only if $\pi$ is a solution to the primal problem, and $\left(  u,v\right)  $ is a solution to the dual problem.

---


Back to the workers / firms interpretation and assume for now that workers are indifferent between any two firms that offer the same salary. We argue that $u\left(  x\right)  $ can be interpreted as the equilibrium wage of worker $x$, while $v\left(  y\right)  $ can be interpreted as the equilibrium profit of firm $y$. Indeed:

---
**Proposition**
If $\left(u,v\right)$ is a solution to the dual of the Kantorovich problem, then

\begin{align*}
u_{x}  &  =\sup_{y\in\mathcal{Y}}\left(  \Phi_{xy}-v_{y}\right)
\label{conjug1}\\
v_{y}  &  =\sup_{x\in\mathcal{X}}\left(  \Phi_{xy}-u_{x}\right)  .
\label{conjug2}
\end{align*}

---

Therefore, $u_{x}$ can be interpreted as equilibrium wage of worker $x$, and $v_{y}$ as equilibrium profit of firm $y$. In this interpretation, all workers get the same wage at equilibrium.

Assume now that if a worker of type $x$ works for a firm of type $y$ for wage $w_{xy}$, then gets $\alpha_{xy}+w_{xy}$, where $\alpha_{xy}$ is the nonmonetary payoff associated with working with a firm of type $y$. The firm's profit is $\gamma_{xy}-w_{xy}$, where $\gamma_{xy}$ is the economic output.

If an employee of type $x$ matches with a firm of type $y$, they generate joint surplus $\Phi_{xy}$, given by%

\begin{align*}
\Phi_{xy}=\underset{\text{employee's payoff}}{\underbrace{\alpha_{xy}+w_{xy}}}+\underset{\text{firm's payoff}}{\underbrace{\gamma_{xy}-w_{xy}}}=\alpha_{xy}+\gamma_{xy}
\end{align*}

which is independent from $w$.

Workers choose firms which maximize their utility, i.e. solve 

<a name="equivWalrasStable1"></a>
\begin{align*}
u_{x}=\max_{y}\left\{  \alpha_{xy}+w_{xy}\right\}
\end{align*}

and $u_{x}=\alpha_{xy}+w_{xy}$ if $x$ and $y$ are matched. Similarly, the indirect payoff vector of firms is

<a name="equivWalrasStable2"></a>
\begin{align*}
v_{y}=\max_{x}\left\{  \gamma_{xy}-w_{xy}\right\}
\end{align*}

and, again, $v_{y}=\gamma_{xy}-w_{xy}$ if $x$ and $y$ are matched.

As a result,

\begin{align*}
u_{x}+v_{y}\geq\alpha_{xy}+\gamma_{xy}=\Phi_{xy}
\end{align*}

and equality holds if $x$ and $y$ are matched. Thus, if $w_{xy}$ is an equilibrium wage, then the triple $\left(  \pi,u,v\right)$ where $\pi$ is the corresponding matching, and $u_{x}$ and $v_{y}$ are defined by [this](#equivWalrasStable1) and [this](#equivWalrasStable2) defines a stable outcome.

Conversely, let $\left(\pi,u,v\right)$ be a stable outcome. Then let $\bar{w}_{xx}$ and \b{w}$_{xy}$ be defined by

\begin{align*}
\bar{w}_{xy}=u_{x}-\alpha_{xy}\text{ and \b{w}}_{xy}=\gamma_{xy}-v_{y}.
\end{align*}

One has $\bar{w}_{xy}\geq$\b{w}$_{xy}$. Any $w_{xy}$ such that $\bar{w}_{xy}\geq w_{xy}\geq$\b{w}$_{xy}$ is an equilibrium wage. Indeed, $\pi_{xy}>0$ implies $\bar{w}_{xy}=$\b{w}$_{xy}$, thus [this](#equivWalrasStable1) and [this](#equivWalrasStable2) hold. Given $u$ and $v$, $w_{xy}$ is uniquely defined on the equilibrium path (ie. when $x$ and $y$ are such that $\pi_{xy}>0$), but there are multiple choices of $w$ outside the equilibrium path.

Note that all workers of the same type get the same indirect utility, but not necessarly the same wage.

## Application

We postulate that the form of the surplus function is
\begin{align*}
\Phi_{ij}=x_{i}^{\intercal} Ay_{j}
\end{align*}
where $x_{i}$ and $y_{j}$ are the 10-dimensional characteristics of man $i$ and woman $j$, and the form of $A$, a 10x10 matrix, is given (it is stored in the file `affinitymatrix.csv`). Again, we'll see later how to solve the econometrics problem of estimating $A$.

In [2]:
A = matrix(as.numeric(data_aff[1:nbcar, 2:(nbcar + 1)]), nbcar, nbcar)

sdX = apply(Xvals, 2, sd)
sdY = apply(Yvals, 2, sd)
mX = apply(Xvals, 2, mean)
mY = apply(Yvals, 2, mean)
Xvals = t((t(Xvals) - mX)/sdX)
Yvals = t((t(Yvals) - mY)/sdY)
nobs = dim(Xvals)[1]

Phi = Xvals %*% A %*% t(Yvals)

This problem of computation of the Optimal Assignment Problem, more specifically of $\left(\pi,u,v\right)$, is arguably the most studied problem in Computer Science, and dozens, if not hundreds of algorithms exist, whose running time is polynomial in $\max\left(n,m\right)$, typically a power less than three of the latter.

Famous algorithms include: the Hungarian algorithm (Kuhn-Munkres); Bertsekas' auction algorithm; Goldberg and Kennedy's pseudoflow algorithm. For more on these, see the book by Burkard, Dell'Amico, and Martello, and a
introductory presentation in http://www.assignmentproblems.com/doc/LSAPIntroduction.pdf.

Here, we will show how to solve the problem with the help of a Linear Programming solver used as a black box; our challenge here will be to carefully set up the constraint matrix as a sparse matrix in order to let a large scale Linear Programming solvers such as Gurobi recognize and exploit the sparsity of the problem.

Let $\Pi$ and $\Phi$ be the matrices with typical elements $\left(\pi_{xy}\right)  $ and $\left(  \Phi_{xy}\right)  $. We let $p$, $q$, $u$,$v$, and $1$ the column vectors with entries $\left(  p_{x}\right)$, $\left(  q_{y}\right)  $, $\left(  u_{x}\right)  $, $\left(  v_{y}\right)$, and $1$, respectively. The optimal assignment problem

\begin{align*}
\max_{\pi\geq0}  &  \sum_{xy}\pi_{xy}\Phi_{xy} \\
s.t.~  &  \sum_{y\in\mathcal{Y}}\pi_{xy}=p_{x}~\left[  u_{x}\right]\\
&  \sum_{x\in\mathcal{X}}\pi_{xy}=q_{y}~\left[  v_{y}\right] 
\end{align*}

Can be rewritten writes using matrix algebra as

\begin{align*}
&  \max_{\Pi\geq0}Tr\left(  \Pi^{\prime}\Phi\right) \\
&  \Pi1_{M}=p\\
&  1_{N}^{\prime}\Pi=q^{\prime}.
\end{align*}

We need to convert matrices into vectors; this can be done for instance by stacking the columns of $\Pi$ into a single column vector. This operation is called *vectorization*, which we will denote

\begin{align*}
vec\left(  A\right)  ,
\end{align*}

which reshapes a $N\times M$ matrix into a $nm\times1$ vector. In `R`, this is
implemented by `c(A)`; in Matlab, by `A(:)`.

The objective function rewrites as

\begin{align*}
vec\left(  \Phi\right)^{\prime}vec\left(  \Pi\right)  .
\end{align*}

In [3]:
obj = c(Phi)

Recall that if $A$ is a $M\times p$ matrix and $B$ a $N\times q$ matrix, then the Kronecker product $A\otimes B$ of $A$ and $B$ is a $mn\times pq$ matrix such that

\begin{align*}
vec\left(  BXA^{\prime}\right)  =\left(  A\otimes B\right)  vec\left(
X\right). \label{VecAndKronecker}
\end{align*}

In R, $A\otimes B$ is implemented by `kronecker(A,B)`; in Matlab, by `kron(A,B)`.

The first constraint $I_{N}\Pi1_{M}=p$, vectorizes therefore as

\begin{align*}
\left(  1_{M}^{\prime}\otimes I_{N}\right)  vec\left(  \Pi\right)  =vec\left(
p\right),
\end{align*}

and similarly, the second constraint $1_{N}^{\prime}\Pi I_{M}=q^{\prime}$, vectorizes as

\begin{align*}
\left(  I_{M}\otimes1_{N}^{\prime}\right)  vec\left(  \Pi\right)  =vec\left(
q\right)  .
\end{align*}

Note that the matrix $1_{M}^{\prime}\otimes I_{N}$ is of size $N\times NM$, and the matrix $I_{M}\otimes1_{N}^{\prime}$ is of size $M\times NM$; hence the full matrix involved in the left-hand side of the constraints is of size $\left(  N+M\right)  \times NM$. In spite of its large size, this matrix is *sparse*. In `R`, the identity matrix $I_{N}$ is coded as `sparseMatrix(1:N,1:N)`, in Matlab as `Speye(N)`.

In [4]:
N = dim(Phi)[1]
M = dim(Phi)[2]

A1 = kronecker(matrix(1, 1, M), sparseMatrix(1:N, 1:N))
A2 = kronecker(sparseMatrix(1:M, 1:M), matrix(1, 1, N))
A = rbind2(A1, A2)

p = rep(1/nobs, nobs)
q = rep(1/nobs, nobs)
d = c(p, q)

Setting $z=vec\left(  \Pi\right)$, the Linear Programming problem then becomes

\begin{align*}
&  \max_{z\geq0}vec\left(  \Phi\right)  ^{\prime}z\label{LPvectorized}\\
s.t.~  &  \left(  1_{M}^{\prime}\otimes I_{N}\right)  z=vec\left(  p\right)
\nonumber\\
&  \left(  I_{M}\otimes1_{N}^{\prime}\right)  z=vec\left(  q^{\prime}\right)
\nonumber
\end{align*}

which is ready to be passed on to a linear programming solver such as Gurobi.

A LP solver typically computes programs of the form

\begin{align*}
&  \max_{z\geq0}c^{\prime}z\label{standardLP}\\
&  s.t.~Az=d.\nonumber
\end{align*}

In [5]:
result = gurobi(list(A = A, obj = obj, modelsense = "max", rhs = d, sense = "="), 
    params = list(OutputFlag = 0))
if (result$status == "OPTIMAL") {
    pi = matrix(result$x, nrow = N)
    u = result$pi[1:N]
    v = result$pi[(N + 1):(N + M)]
    val = result$objval
} else {
    stop("optimization problem with Gurobi.")
}

print(paste0("Value of the problem (Gurobi) = ", val))
print(u[1:10])
print(v[1:10])

[1] "Value of the problem (Gurobi) = 1.70388302245657"
 [1] 1.922803 1.286440 2.351599 3.030279 3.741377 2.725222 1.252313 1.988384
 [9] 1.445145 1.525087
 [1] -0.7409078 -0.9616074  0.6039734 -0.2880301 -1.1140921 -0.1014630
 [7] -0.6873566  0.5351975 -0.5838891 -0.1083689


Who does man $1$ match with?

In [6]:
print(paste('Woman', which(pi[1,]!=0)))

[1] "Woman 576"
